In [2]:
#pip install rdkit docx py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolAlign
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import Pt, Inches
import os
import py3Dmol


# === CONFIGURA TUS RUTAS ===
antibioticos = {
    "imipenem_original": r"C:\Tareas004\TERCERPARCIAL\imipenem.mol",
    "imipenem_mod1": r"C:\Tareas004\TERCERPARCIAL\imipenemMETOXI.mol",
    "imipenem_mod2": r"C:\Tareas004\TERCERPARCIAL\imipenemCONOXIGENO.mol",
    "imipenem_mod3": r"C:\Tareas004\TERCERPARCIAL\imipenemACIDOBORICO.mol",
    "meropenem_original": r"C:\Tareas004\TERCERPARCIAL\meropenem.mol",
    "meropenem_mod1": r"C:\Tareas004\TERCERPARCIAL\meropenemMETOXI.mol",
    "meropenem_mod2": r"C:\Tareas004\TERCERPARCIAL\meropenemCH2.mol"
}

# === Cargar moléculas y generar conformadores con semilla fija ===
def cargar_y_preparar(ruta):
    mol = Chem.MolFromMolFile(ruta, removeHs=False)
    mol = Chem.AddHs(mol)
    if mol is None:
        raise ValueError(f"No se pudo cargar {ruta}")
    AllChem.EmbedMolecule(mol, randomSeed=42)
    AllChem.MMFFOptimizeMolecule(mol)
    return mol

mol_dict = {nombre: cargar_y_preparar(ruta) for nombre, ruta in antibioticos.items()}

# === SMARTS del núcleo beta-lactámico ===
core_smarts = 'C1C(=O)N(C1)'
core = Chem.MolFromSmarts(core_smarts)

# === Calcular RMSD con alineamiento por núcleo ===
def rmsd_con_core(mol_ref, mol_obj, core):
    match_ref = mol_ref.GetSubstructMatch(core)
    match_obj = mol_obj.GetSubstructMatch(core)
    if not match_ref or not match_obj:
        raise RuntimeError("No se encontró el núcleo común en una de las moléculas")
    atom_map = list(zip(match_obj, match_ref))
    rmsd = rdMolAlign.AlignMol(mol_obj, mol_ref, atomMap=atom_map)
    return rmsd

# === Calcular RMSD reales y almacenarlos ===
rmsd_dict = {}
for fam in ["imipenem", "meropenem"]:
    ref = mol_dict[f"{fam}_original"]
    print(f"\nRMSD respecto a {fam}_original:")
    for nombre, mol in mol_dict.items():
        if nombre.startswith(fam) and nombre != f"{fam}_original":
            try:
                rmsd = rmsd_con_core(ref, mol, core)
                rmsd_dict[nombre] = rmsd
                print(f"  {nombre}: RMSD = {rmsd:.3f} Å")
            except Exception as e:
                print(f"  {nombre}: Error - {e}")

# === Crear documento Word ===
doc = Document()
doc.add_heading('Informe de Análisis Estructural: Comparación de Antibióticos Modificados', 0)

doc.add_heading('Evaluación mediante RMSD', level=1)
doc.add_paragraph(
    "El análisis estructural de las modificaciones propuestas para los antibióticos imipenem y meropenem se realizó "
    "mediante el cálculo del RMSD (desviación cuadrática media) entre cada antibiótico modificado y su respectiva "
    "estructura original. El RMSD es una métrica cuantitativa que permite evaluar cuánto cambia la conformación espacial "
    "de una molécula en comparación con una referencia."
)

doc.add_heading('Resultados', level=1)

# --- Imipenem ---
doc.add_heading('Imipenem:', level=2)
doc.add_paragraph(f"- imipenem_mod1: RMSD = {rmsd_dict['imipenem_mod1']:.3f} Å")
doc.add_paragraph(f"- imipenem_mod2: RMSD = {rmsd_dict['imipenem_mod2']:.3f} Å")
doc.add_paragraph(f"- imipenem_mod3: RMSD = {rmsd_dict['imipenem_mod3']:.3f} Å")
doc.add_paragraph(
    "Las modificaciones agregando un grupo metoxi (mod1) y reemplazando el azufre por oxígeno (mod2) presentan RMSD muy bajos (< 0.1 Å), "
    "lo que indica que mantienen prácticamente la misma conformación que el imipenem original. Estas modificaciones son estructuralmente viables. "
    "En cambio, la modificación agregando un ácido bórico (mod3) presenta un RMSD más alto, lo que sugiere una mayor alteración estructural."
)

# --- Meropenem ---
doc.add_heading('Meropenem:', level=2)
doc.add_paragraph(f"- meropenem_mod1: RMSD = {rmsd_dict['meropenem_mod1']:.3f} Å")
doc.add_paragraph(f"- meropenem_mod2: RMSD = {rmsd_dict['meropenem_mod2']:.3f} Å")
doc.add_paragraph(
    "La modificación agregando un grupo metoxi (mod1) mantiene una conformación muy similar a la original, por lo que es estructuralmente estable. "
    "La modificación agregando un grupo de metilo (mod2) tiene un RMSD más alto, así que presenta un cambio más notorio en su estructura espacial, "
    "lo cual podría comprometer su capacidad de unión o reconocimiento por parte de la enzima NDM-1."
)

# --- Conclusiones ---
doc.add_heading('Conclusiones', level=1)
doc.add_paragraph(
    "Las modificaciones imipenem_mod1, imipenem_mod2 y meropenem_mod1 son las más prometedoras desde un punto de vista estructural, "
    "ya que conservan la geometría del antibiótico original. Las modificaciones con RMSD más elevados (imipenem_mod3 y meropenem_mod2) "
    "podrían requerir un análisis funcional adicional para determinar si su alteración espacial compromete la actividad antibiótica "
    "o mejora su evasión de la enzima NDM-1."
)

# === Imagen (firma) ===
ruta_imagen = r"C:\Tareas002\SegundoParcial\Firma.eje1.jpg"
if os.path.exists(ruta_imagen):
    p_img = doc.add_paragraph()
    p_img.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run_img = p_img.add_run()
    run_img.add_picture(ruta_imagen, width=Inches(2.0))
else:
    doc.add_paragraph("[Firma o imagen no disponible]", style='Intense Quote')

# === Firma textual ===
def agregar_firma(paragraph, texto, bold=False):
    run = paragraph.add_run(texto)
    run.bold = bold
    run.font.size = Pt(10)
    run.font.name = 'Tahoma'

p_nombre = doc.add_paragraph()
p_nombre.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
agregar_firma(p_nombre, "INGENIERA ANABEL ROSSIO PLATA BURGOA", bold=True)

p_ci = doc.add_paragraph()
p_ci.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
agregar_firma(p_ci, "CI: 12830436 LP")

# === Guardar documento ===
doc.save("Informe_RMSD_Antibioticos.docx")
print("✅ Informe Word generado con éxito.")


# === Visualización sencilla con py3Dmol de un grupo 
view = py3Dmol.view(width=600, height=400)
colores = ["blue", "green", "red", "orange"]
for i, nombre in enumerate([f"imipenem_original", "imipenem_mod1", "imipenem_mod2", "imipenem_mod3"]):
    pdb_block = Chem.MolToPDBBlock(mol_dict[nombre])
    view.addModel(pdb_block, "pdb")
    view.setStyle({'model': i}, {'stick': {'color': colores[i]}})

# Agregar título
view.addLabel("Comparación de estructuras de Imipenem", 
             {'fontSize': 14, 'fontColor': 'black', 'backgroundColor': 'white'},
             {'x': 200, 'y': 10})
view.zoomTo()
view.show()

view = py3Dmol.view(width=600, height=400)
colores = ["blue", "green", "red", "orange"]
for i, nombre in enumerate([f"meropenem_original", "meropenem_mod1", "meropenem_mod2"]):
    pdb_block = Chem.MolToPDBBlock(mol_dict[nombre])
    view.addModel(pdb_block, "pdb")
    view.setStyle({'model': i}, {'stick': {'color': colores[i]}})

# Agregar título
view.addLabel("Comparación de estructuras de Meropenem", 
             {'fontSize': 14, 'fontColor': 'black', 'backgroundColor': 'white'},
             {'x': 200, 'y': 10})

view.zoomTo()
view.show()



RMSD respecto a imipenem_original:
  imipenem_mod1: RMSD = 0.055 Å
  imipenem_mod2: RMSD = 0.054 Å
  imipenem_mod3: RMSD = 0.357 Å

RMSD respecto a meropenem_original:
  meropenem_mod1: RMSD = 0.015 Å
  meropenem_mod2: RMSD = 0.018 Å
✅ Informe Word generado con éxito.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [3]:
#pip install MDAnalysis nglview numpy python-docx rdkit
#pip install ipywidgets
import MDAnalysis as mda
from MDAnalysis.core.universe import Merge
import nglview as nv
import numpy as np
from docx import Document
from docx.shared import Pt, Inches 
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from rdkit import Chem
from rdkit.Chem import AllChem
import warnings
warnings.filterwarnings('ignore')


# --- CONFIGURA TUS RUTAS ---

ruta_enzima = r"C:\Tareas004\TERCERPARCIAL\ndm1_fixed.pdb"

antibioticos = {
    "imipenem_original": r"C:\Tareas004\TERCERPARCIAL\imipenem.mol",
    "imipenem_mod1": r"C:\Tareas004\TERCERPARCIAL\imipenemMETOXI.mol",
    "imipenem_mod2": r"C:\Tareas004\TERCERPARCIAL\imipenemCONOXIGENO.mol",
    "imipenem_mod3": r"C:\Tareas004\TERCERPARCIAL\imipenemACIDOBORICO.mol",
    "meropenem_original": r"C:\Tareas004\TERCERPARCIAL\meropenem.mol",
    "meropenem_mod1": r"C:\Tareas004\TERCERPARCIAL\meropenemMETOXI.mol",
    "meropenem_mod2": r"C:\Tareas004\TERCERPARCIAL\meropenemCH2.mol"
}

# --- Funciones ---

def cargar_antibiotico(mol_path):
    

    mol = Chem.MolFromMolFile(mol_path)
    if mol is None:
        raise ValueError(f"No se pudo cargar {mol_path}")
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, randomSeed=42)
    AllChem.MMFFOptimizeMolecule(mol)

    pdb_block = Chem.MolToPDBBlock(mol)
    temp_pdb = mol_path.replace('.mol', '_temp.pdb')
    with open(temp_pdb, 'w') as f:
        f.write(pdb_block)

    u = mda.Universe(temp_pdb)
    return u

def crear_complejo(u_enz, u_lig, nombre_salida):
    complejo = Merge(u_enz.atoms, u_lig.atoms)
    complejo.atoms.write(nombre_salida)
    return nombre_salida

def calcular_contactos(u_complejo, resname_ligando, cutoff=4.0):
    prot = u_complejo.select_atoms("protein")
    lig = u_complejo.select_atoms(f"resname {resname_ligando}")

    if len(lig) == 0:
        print(f"¡No se encontró ligando con resname {resname_ligando}!")
        return set()

    contacts = set()
    for a_prot in prot.atoms:
        for a_lig in lig.atoms:
            dist = np.linalg.norm(a_prot.position - a_lig.position)
            if dist <= cutoff:
                contacts.add((a_prot.resid, a_prot.resname, a_lig.resid, a_lig.resname))
    return contacts

def comparar_contactos(contacts_orig, contacts_mod):
    comunes = contacts_orig.intersection(contacts_mod)
    perdidos = contacts_orig.difference(contacts_mod)
    nuevos = contacts_mod.difference(contacts_orig)
    return comunes, perdidos, nuevos



def crear_reporte_word(resultados_contactos, filename="reporte_resultados.docx"):
    doc = Document()
    doc.add_heading("Reporte de Interacciones Antibióticos - Enzima NDM-1", level=1)
    
    # Explicación general sobre contactos y comparación
    intro = (
        "En este reporte se presentan los resultados del análisis de contactos moleculares "
        "entre la enzima bacteriana NDM-1 y diferentes antibióticos, tanto en su forma original "
        "como en varias modificaciones químicas.\n\n"
        "Un 'contacto' se define como un par de átomos, uno perteneciente a la proteína (enzima) "
        "y otro al antibiótico, que están a una distancia menor o igual a 4 Å, lo que indica una "
        "interacción cercana potencialmente relevante para la actividad bioquímica.\n\n"
        "Las comparaciones entre antibióticos modificados y su versión original muestran:\n"
        "- Contactos comunes: interacciones que se mantienen tras la modificación, indicativo de "
        "que la unión al sitio activo se conserva.\n"
        "- Contactos perdidos: interacciones presentes en el original pero ausentes en la modificación, "
        "lo que puede sugerir pérdida de afinidad o alteración en la unión.\n"
        "- Contactos nuevos: interacciones que aparecen solo en la modificación, que pueden ser "
        "nuevas uniones beneficiosas o no deseadas.\n\n"
        "Este análisis permite evaluar la viabilidad estructural y funcional de las modificaciones "
        "realizadas a los antibióticos."
    )
    p_intro = doc.add_paragraph(intro)
    p_intro.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
    
    doc.add_paragraph(" ")

    # Resultados por antibiótico (contactos totales)
    for ab_name, contacts in resultados_contactos.items():
        doc.add_heading(f"Antibiótico: {ab_name}", level=2)
        p = doc.add_paragraph(f"Número total de contactos encontrados: {len(contacts)}")
        p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        doc.add_paragraph(" ")

    doc.add_heading("Comparaciones de contactos contra originales", level=1)

    # Separar originales
    originales = {k:v for k,v in resultados_contactos.items() if 'original' in k}
    for orig_name, orig_contacts in originales.items():
        doc.add_heading(f"Comparaciones para {orig_name}", level=2)
        for mod_name, mod_contacts in resultados_contactos.items():
            if mod_name.startswith(orig_name.replace('_original','')) and mod_name != orig_name:
                comunes, perdidos, nuevos = comparar_contactos(orig_contacts, mod_contacts)
                doc.add_paragraph(f"Comparación: {mod_name} vs {orig_name}")
                doc.add_paragraph(f"  Contactos comunes (mantienen la interacción): {len(comunes)}")
                doc.add_paragraph(f"  Contactos perdidos (interacciones que ya no están): {len(perdidos)}")
                doc.add_paragraph(f"  Contactos nuevos (nuevas interacciones): {len(nuevos)}")
                doc.add_paragraph(" ")

    # --- Agregar firma textual ---
    def agregar_firma(paragraph, texto, bold=False):
        run = paragraph.add_run(texto)
        run.bold = bold
        run.font.size = Pt(10)
        run.font.name = 'Tahoma'

    doc.add_paragraph(" ")  # Espacio antes de la firma
    ruta_imagen = r"C:\Tareas002\SegundoParcial\Firma.eje1.jpg"  
    
    try:
        # Añadir imagen centrada
        paragraph_img = doc.add_paragraph()
        paragraph_img.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        paragraph_img.paragraph_format.space_after = Inches(0)  
        run_img = paragraph_img.add_run()
        run_img.add_picture(ruta_imagen, width=Inches(2.0))  
    except FileNotFoundError:
        print(f"Advertencia: No se encontró la imagen en {ruta_imagen}")
        doc.add_paragraph("[Firma o imagen no disponible]", style='Intense Quote')    
    

    p_nombre = doc.add_paragraph()
    p_nombre.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    agregar_firma(p_nombre, "INGENIERA ANABEL ROSSIO PLATA BURGOA", bold=True)

    p_ci = doc.add_paragraph()
    p_ci.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    agregar_firma(p_ci, "CI: 12830436 LP")

    doc.save(filename)
    print(f"Reporte guardado en: {filename}")

 


# --- Ejecutar análisis ---

u_enzima = mda.Universe(ruta_enzima)
resname_ligando = "UNL"  

contactos_resultados = {}

for nombre, ruta_mol in antibioticos.items():
    print(f"Procesando {nombre} ...")
    u_lig = cargar_antibiotico(ruta_mol)
    nombre_complejo = f"complejo_{nombre}.pdb"
    crear_complejo(u_enzima, u_lig, nombre_complejo)

    u_complejo = mda.Universe(nombre_complejo)
    contacts = calcular_contactos(u_complejo, resname_ligando)
    print(f"Contactos encontrados para {nombre}: {len(contacts)}")
    contactos_resultados[nombre] = contacts

print("\n--- Comparación de contactos ---\n")

# Imprimir comparación por consola también
original_imipenem = contactos_resultados.get("imipenem_original", set())
for nombre, contacts_mod in contactos_resultados.items():
    if nombre.startswith("imipenem") and nombre != "imipenem_original":
        comunes, perdidos, nuevos = comparar_contactos(original_imipenem, contacts_mod)
        print(f"Comparación {nombre} vs imipenem_original:")
        print(f"  Contactos comunes: {len(comunes)}")
        print(f"  Contactos perdidos: {len(perdidos)}")
        print(f"  Contactos nuevos: {len(nuevos)}")
        print()

original_meropenem = contactos_resultados.get("meropenem_original", set())
for nombre, contacts_mod in contactos_resultados.items():
    if nombre.startswith("meropenem") and nombre != "meropenem_original":
        comunes, perdidos, nuevos = comparar_contactos(original_meropenem, contacts_mod)
        print(f"Comparación {nombre} vs meropenem_original:")
        print(f"  Contactos comunes: {len(comunes)}")
        print(f"  Contactos perdidos: {len(perdidos)}")
        print(f"  Contactos nuevos: {len(nuevos)}")
        print()

# Crear reporte Word con los resultados
crear_reporte_word(contactos_resultados)

# Visualización interactiva del último complejo generado (ejemplo imipenem_mod1)
u_vis = mda.Universe(f"complejo_imipenem_mod1.pdb")
view = nv.show_mdanalysis(u_vis)
view.center(selection="resid 120")  # Ajusta residuo para centrar en sitio activo
view


Procesando imipenem_original ...
Contactos encontrados para imipenem_original: 22
Procesando imipenem_mod1 ...
Contactos encontrados para imipenem_mod1: 19
Procesando imipenem_mod2 ...
Contactos encontrados para imipenem_mod2: 21
Procesando imipenem_mod3 ...
Contactos encontrados para imipenem_mod3: 23
Procesando meropenem_original ...
Contactos encontrados para meropenem_original: 25
Procesando meropenem_mod1 ...
Contactos encontrados para meropenem_mod1: 27
Procesando meropenem_mod2 ...
Contactos encontrados para meropenem_mod2: 25

--- Comparación de contactos ---

Comparación imipenem_mod1 vs imipenem_original:
  Contactos comunes: 17
  Contactos perdidos: 5
  Contactos nuevos: 2

Comparación imipenem_mod2 vs imipenem_original:
  Contactos comunes: 18
  Contactos perdidos: 4
  Contactos nuevos: 3

Comparación imipenem_mod3 vs imipenem_original:
  Contactos comunes: 21
  Contactos perdidos: 1
  Contactos nuevos: 2

Comparación meropenem_mod1 vs meropenem_original:
  Contactos comune

NGLWidget()